This notebook is designed to create the _benthos_ interpolated maps using `DIVAnd`.      
The data file was prepared by P. Herman (Deltares).

In [15]:
using DIVAnd
using PyPlot
using Proj4
using DelimitedFiles
using PyCall
using Dates
using NCDatasets
include("../scripts/BenthosInterp.jl")
doplot = false
usecartopy = true
writenc = true

true

## Files and directories

In [2]:
figdir = "../product/figures/"
outputdir = "../product/netCDF/"
datadir = "../data/"
datafile = joinpath(datadir, "specs4Diva.csv")
isdir(figdir) ? "Figure directory already exists" : mkpath(figdir)
isdir(outputdir) ? "Output directory already exists" : mkpath(outputdir)
isfile(datafile) ? @info("File already downloaded") : download("https://dox.ulg.ac.be/index.php/s/vNQcvqjW8RzdNBt/download", datafile)

┌ Info: File already downloaded
└ @ Main In[2]:7


In [3]:
domain = [-16., 9., 45., 66.]; # [West East South North]
Δlon = 0.1
Δlat = 0.1

0.1

## Prepare mask
### Interpolation grid

In [4]:
longrid = domain[1]:Δlon:domain[2]
latgrid = domain[3]:Δlat:domain[4]

45.0:0.1:66.0

### Download bathymetry file

In [5]:
bathname = joinpath(datadir, "gebco_30sec_4.nc")
if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/RSwm4HPHImdZoQP/download", bathname)
else
    @info("Bathymetry file already downloaded")
end

┌ Info: Bathymetry file already downloaded
└ @ Main In[5]:5


### Read bathymetry

In [6]:
bx, by, b = load_bath(bathname, true, longrid, latgrid)
@show size(b)

if doplot
    fig = PyPlot.figure()
    ax = PyPlot.subplot(111)
    pcolor(bx,by,b', vmin=0., cmap=PyPlot.cm.gist_earth); 
    colorbar(orientation="vertical")
    title("Depth (m)")
    savefig(joinpath(figdir, "benthos_bathy.jpg"), dpi=300, bbox_inches="tight")
    show()
end

size(b) = (251, 211)


### Metrics

In [7]:
_, (pm, pn),(xi, yi) = DIVAnd.DIVAnd_rectdom(longrid, latgrid);
xi, yi, mask = DIVAnd.load_mask(bathname, true, longrid, latgrid, 0.0);
xx, yy = ndgrid(xi, yi);

## Interpolation
### Data reading

Loop on all the species: 
1. read the data
2. compute the heatmaps and 
3. derive the probability field as:
```
d = npre * dens2 / (npre * dens_pre + nabs * dens_abs)
```
where 
* dens_pre is the heatmap obtained with the presence data only; 
* dens_abs is the heatmap obtained with the absence data only. 

The reason for this equation is that the heatmap are computed so that their integral over the domain is 1, whatever the number of observations. 

In [8]:
# Set correlation length
Lvalues = [0.01, 0.05, 0.1, 0.5, 1.]
L = 0.1

0.1

In [9]:
include("../scripts/BenthosInterp.jl")

write_nc_error

In [16]:
speciesnamelist = get_species_list(datafile)
for species in speciesnamelist

    speciesslug = get_species_slug(String(species))
    
    @info(speciesslug)
    @info("Working on species $(species)");
    lon_pre, lat_pre, lon_abs, lat_abs = read_coords_species(datafile, species);
    npre = length(lon_pre)
    nabs = length(lon_abs)

    @info("Number of presence: $(npre), number of absence: $(nabs)")
    @info("Computing heatmaps")

    ii = 0
    

    ii += 1
    dens_pre, LHM2, LCV2, LSCV2 = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon_pre, lat_pre), ones(npre), L);
    dens_abs, LHM3, LCV3, LSCV3 = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon_abs, lat_abs), ones(nabs), L);


    d = npre .* dens_pre ./ (npre .* dens_pre .+ nabs .* dens_abs);

    @info("Computing error field with CPME")
    lon = [lon_pre ; lon_abs]
    lat = [lat_pre ; lat_abs]

    cpme = DIVAnd_cpme(mask, (pm, pn), (xx, yy), (lon, lat), 
        ones(length(lon)), 0.5, 5.);

    if doplot
        plot_heatmap(longrid, latgrid, d, lon_pre, lat_pre, lon_abs, lat_abs,
            "$(species): probability", figname=joinpath(figdir, "$(speciesslug)_density.png"), 
            usecartopy=usecartopy)            
        #plot_error(longrid, latgrid, cpme, "$(species)", 
        #    joinpath(figdir, "$(speciesslug)_error.png"))
    end   

    if writenc
        @info("Creating the netCDF file with results")
        create_nc_results(joinpath(outputdir, "$(speciesslug)_density.nc"), 
            longrid, latgrid, d, String(species));

        @info("Adding error field to netCDF file")
        write_nc_error(joinpath(outputdir, "$(speciesslug)_density.nc"), cpme);
    end


end

┌ Info: Abra_alba
└ @ Main In[16]:6
┌ Info: Working on species Abra_alba
└ @ Main In[16]:7
┌ Info: Column index for Abra_alba: 5
└ @ Main /home/ctroupin/Projects/EMODnet/EMODnet-Biology-Benthos-Interpolated-Maps/scripts/BenthosInterp.jl:21
┌ Info: Number of presence: 14068, number of absence: 65977
└ @ Main In[16]:12
┌ Info: Computing heatmaps
└ @ Main In[16]:13
┌ Warning: Data array size (14068,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/LkI0S/src/DIVAnd_heatmap.jl:48
┌ Warning: Data array size (65977,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/LkI0S/src/DIVAnd_heatmap.jl:48
┌ Info: Computing error field with CPME
└ @ Main In[16]:27
┌ Info: Creating the netCDF file with results
└ @ Main In[16]:43
┌ Info: Adding error field to net

┌ Info: Computing error field with CPME
└ @ Main In[16]:27
┌ Info: Creating the netCDF file with results
└ @ Main In[16]:43
┌ Info: Adding error field to netCDF file
└ @ Main In[16]:47
┌ Info: Littorina_littorea
└ @ Main In[16]:6
┌ Info: Working on species Littorina_littorea
└ @ Main In[16]:7
┌ Info: Column index for Littorina_littorea: 15
└ @ Main /home/ctroupin/Projects/EMODnet/EMODnet-Biology-Benthos-Interpolated-Maps/scripts/BenthosInterp.jl:21
┌ Info: Number of presence: 1285, number of absence: 67275
└ @ Main In[16]:12
┌ Info: Computing heatmaps
└ @ Main In[16]:13
┌ Warning: Data array size (67275,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/LkI0S/src/DIVAnd_heatmap.jl:48
┌ Info: Computing error field with CPME
└ @ Main In[16]:27
┌ Info: Creating the netCDF file with results
└ @ Main In[16]:43
┌ Info: Adding error field to netCDF file
└ @ Main In[16]:47

Create a new field that takes into account the error field:
* if error is zero, we take the gridded field as it is;
* if error is close to 1, it means we don't have observations so the new field is zero (assuming that we don't have observations means there is nothing to observe);
* if error is between 0 and 1, we calculate the field as 

In [11]:
newfield = (1. .- cpme) .* d;

LoadError: UndefVarError: cpme not defined

## Variable correlation length